#CM4709 Computer Vision
#Lab 07 Image Segmentation

##Aims
* Use YOLO to pefrom image segmentation.

Note: The original lab used [Mask RCNN](https://github.com/matterport/Mask_RCNN) in the [PixelLib](https://github.com/ayoolaolafenwa/PixelLib) library.
However, as software have evolved, the current library versions do not seem to work with Mask RCNN and the older version of Tensorflow is not available in Colab anymore.
As a result, we changed to use YOLO. The whole idea is to illustrate how to do image segmentation using in an
existing library.

##Mounting GoogleDrive to Colab Runtime

As usual, you will need some testing images. It is faster to upload them to GoogleDrive and then mount it to your Colab runtime.

In the following code, I use the path `/content/drive` as the mounting point as Colab also has a convenient button mounting to this path.

In [ ]:
from google.colab import drive

gDriveMountPoint='/content/gDrive'  #mounting point
drive.mount(gDriveMountPoint)       #mount GoogleDrive

##Testing Images

You are advised to organise your files in GoogleDrive. e.g. Create a `Lab07` folder in GoogleDrive to store the notebook file of Lab 07, and a `Lab07/data` folder for the images.

1. Download the testing images in Moodle. Optionally, use your own testing images.
1. Upload testing images to your GoogleDrive space.

The following code shows a testing image using OpenCV.
* Note: Modify the filename and image path accordingly.


In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

#image filename and path
#
filename='test.jpg'
imagePath=gDriveMountPoint+'/MyDrive/cm4709/Lab07/data'
imageFile=imagePath+'/'+filename

#show image
#
img=cv.imread(imageFile)
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

##Image Segmentation Using YOLO

Interesting enough, YOLO now also has the image segmentation function.
Program code in this part is mostly from [this web page](https://dev.to/andreygermanov/how-to-implement-instance-segmentation-using-yolov8-neural-network-3if9).

To use YOLO for the task, we need to install the Ultalytics library:

In [ ]:
#install Ultralytic library

!pip install ultralytics

##Perform Image Segmentation Using YOLO

For image segementation, we will need a different weight file trained for the task.
Apart from this, the Python code is very similar to object detection.
After calling the [`predict(...)` function](https://docs.ultralytics.com/modes/predict/#inference-arguments), what remains is to process the result.

In [ ]:
from ultralytics import YOLO

#
#path to input image
#these lines are already executed above
#just here for reference
#
#filename='test.jpg'
#imagePath=gDriveMountPoint+'/MyDrive/cm4709/Lab07/data'
#imageFile=imagePath+'/'+filename

#
#download weights for image segmentation
#
model = YOLO("yolo11m-seg.pt")      #load YOLOv11m model weights for image segmentation
results = model.predict(imageFile)  #predict on 1 image

##Processing the Masks

The prediction [result](https://docs.ultralytics.com/modes/predict/#inference-arguments) contains a number of [masks](https://docs.ultralytics.com/modes/predict/#masks).
The following code shows the number of masks detected in the image:

In [ ]:
result = results[0]                 #take 1st result as we have only 1 image
masks=result.masks                  #get masks predicted
print('No. of masks: ',len(result))

##Showing the Masks as Polygons

Each mask contains list of x,y coordinates in floating point numbers.
By converting the list of coordinates into integers, we can draw out each mask as a polygon using OpenCV.

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy
import random

#
#draw masks as polygons
#
img=cv.imread(imageFile)
for mask in masks:
  xy=mask.xy[0].astype(int)   #convert float to int
  colour=(random.randint(0,255),random.randint(0,255),random.randint(0,255))  #create a random BGR colour for drawing
  cv.polylines(img,[xy],isClosed=True,color=colour,thickness=3)               #draw mask as a polygon

#
#show image with masks drawn
#
plt.figure(figsize=(10,10))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))